## **Word2Vec for sentyment analysis.**


*   I've imported the dataset from google drive (https://drive.google.com/drive/folders/0Bz8a_Dbh9Qhbfll6bVpmNUtUcFdjYmF2SEpmZUZUcVNiMUw1TWN6RDV3a0JHT3kxLVhVR2M?resourcekey=0-TLwzfR2O-D2aPitmn5o9VQ)




In [5]:
!pip install -q gdown

In [6]:
fileid = '0Bz8a_Dbh9QhbZVhsUnRWRDhETzA'
outputname = 'amazon_review_full_csv.tar.gz'

!gdown --id {fileid} -O {outputname}

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=0Bz8a_Dbh9QhbZVhsUnRWRDhETzA
From (redirected): https://drive.google.com/uc?id=0Bz8a_Dbh9QhbZVhsUnRWRDhETzA&confirm=t&uuid=e7c7f9cf-c6b4-4f72-b642-32a7f7c32717
To: /content/amazon_review_full_csv.tar.gz
100% 644M/644M [00:09<00:00, 69.3MB/s]


In [7]:
import tarfile
with tarfile.open(outputname, "r:gz") as tar:
  tar.extractall("Dataset")

In [8]:
!ls -R Dataset/

Dataset/:
amazon_review_full_csv

Dataset/amazon_review_full_csv:
readme.txt  test.csv  train.csv


In [ ]:
!pip install gensim nltk scikit-learn

### **Text Processing:**

1.   Import the libraryes
2.   Both in training and test i merge the colums 'Title' and 'Review' in a new column called 'Text'.
3. Lowercase
4. Remove punctuation and numbers
4. Remove stopwords
6. Lemmatization





In [28]:
import pandas as pd
import string
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download NLTK resources (run only once)
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
df_train = pd.read_csv(
    './Dataset/amazon_review_full_csv/train.csv',
    header=None,
    names=['Rating', 'Title', 'Review'],
    quotechar='"',
    doublequote=True,
    escapechar='\\',
    engine='python',
    encoding='utf-8',
    on_bad_lines='skip'  # Skip rows with parsing errors
)

df_train.head(10)

,Rating,Title,Review
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...
5,5,There's a reason for the price,"There's a reason this CD is so expensive, even..."
6,1,Buyer beware,"This is a self-published book, and if you want..."
7,4,"Errors, but great story",I was a dissapointed to see errors on the back...
8,1,The Worst!,A complete waste of time. Typographical errors...
9,1,Oh please,I guess you have to be a romance novel lover f...


In [23]:
df_test = pd.read_csv(
    './Dataset/amazon_review_full_csv/test.csv',
    header=None,
    names=['Rating', 'Title', 'Review'],
    quotechar='"',
    doublequote=True,
    escapechar='\\',
    engine='python',
    encoding='utf-8',
    on_bad_lines='skip'  # Skip rows with parsing errors
)

df_test.head(10)

,Rating,Title,Review
0,1,mens ultrasheer,"This model may be ok for sedentary types, but ..."
1,4,Surprisingly delightful,This is a fast read filled with unexpected hum...
2,2,"Works, but not as advertised",I bought one of these chargers..the instructio...
3,2,Oh dear,I was excited to find a book ostensibly about ...
4,2,Incorrect disc!,"I am a big JVC fan, but I do not like this mod..."
5,2,Incorrect Disc,"I love the style of this, but after a couple y..."
6,2,DVD menu select problems,I cannot scroll through a DVD menu that is set...
7,3,My 2 y/o grandson loves it!!,This movie with all of its animals really keep...
8,5,A Cookbook Every Baker Should Own,I found a copy of this cookbook at a local use...
9,3,good basic,"The book is a basic ""how to"" book for using so..."


In [24]:
df_train["Text"] = df_train["Title"].fillna("") + " " + df_train["Review"].fillna("")
df_train = df_train[["Text", "Rating"]]
df_train.head(10)

,Text,Rating
0,more like funchuck Gave this to my dad for a g...,3
1,Inspiring I hope a lot of people hear this cd....,5
2,The best soundtrack ever to anything. I'm read...,5
3,Chrono Cross OST The music of Yasunori Misuda ...,4
4,Too good to be true Probably the greatest soun...,5
5,There's a reason for the price There's a reaso...,5
6,"Buyer beware This is a self-published book, an...",1
7,"Errors, but great story I was a dissapointed t...",4
8,The Worst! A complete waste of time. Typograph...,1
9,Oh please I guess you have to be a romance nov...,1


In [25]:
df_test["Text"] = df_test["Title"].fillna("") + " " + df_test["Review"].fillna("")
df_test = df_test[["Text", "Rating"]]
df_test.head(10)

,Text,Rating
0,mens ultrasheer This model may be ok for seden...,1
1,Surprisingly delightful This is a fast read fi...,4
2,"Works, but not as advertised I bought one of t...",2
3,Oh dear I was excited to find a book ostensibl...,2
4,"Incorrect disc! I am a big JVC fan, but I do n...",2
5,"Incorrect Disc I love the style of this, but a...",2
6,DVD menu select problems I cannot scroll throu...,2
7,My 2 y/o grandson loves it!! This movie with a...,3
8,A Cookbook Every Baker Should Own I found a co...,5
9,"good basic The book is a basic ""how to"" book f...",3


In [31]:
# LOAD THE STOPWORDS (FROM NLTK)
stop_words = set(stopwords.words('english'))

# Words that are considered stopwords but i don't want to remove
important_words = {
    "not", "no", "nor", "never", "none", "nobody", "nothing", "neither", "nowhere",
    "don’t", "doesn’t", "didn’t", "can’t", "couldn’t", "won’t", "wouldn’t",
    "shouldn’t", "wasn’t", "weren’t", "isn’t", "aren’t", "hasn’t", "haven’t", "hadn’t",
    "cannot", "without", "hardly", "barely", "rarely", "scarcely"
}
stop_words = stop_words - important_words

def cleaning1(text):
    # Convert all the text to lowercase (optional)
    text = str(text).lower()

    # Replace multiple ! and ? with only one occurrency!
    text = re.sub(r'[!]{2,}', '!', text)  # OK !!! --> ok !
    text = re.sub(r'[?]{2,}', '?', text)  # Are you ok??? --> are you ok?

    # Remove any special characters except for a single ! or ?
    # It avoids emoticons, urls ecc... (WATCH AGAIN LATER)
    text = re.sub(r'[^a-z0-9\s!?]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenize and remove stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]

    return ' '.join(words)

df_train['Text'] = df_train['Text'].apply(cleaning1)
df_test['Text'] = df_test['Text'].apply(cleaning1)


df_train.head(10)

,Text,Rating
0,like funchuck gave dad gag gift directing nuns...,3
1,inspiring hope lot people hear cd need strong ...,5
2,best soundtrack ever anything im reading lot r...,5
3,chrono cross ost music yasunori misuda without...,4
4,good true probably greatest soundtrack history...,5
5,theres reason price theres reason cd expensive...,5
6,buyer beware selfpublished book want know whyr...,1
7,errors great story dissapointed see errors bac...,4
8,worst! complete waste time typographical error...,1
9,oh please guess romance novel lover one not di...,1


#### **A questo punto conviene fare un subset**
- To much data to process


In [33]:
nlp = spacy.load("en_core_web_sm")

#LEMMATIZATION FUNCTION

def lemmatize_text(text):
    #spaCy
    doc = nlp(str(text))
    return " ".join([token.lemma_ for token in doc])

df_train['Text'] = df_train['Text'].apply(lemmatize_text)
df_test['Text'] = df_test['Text'].apply(lemmatize_text)

df_test.head(10)

KeyboardInterrupt: 